# 误差反向传播法

作者：杨岱川

时间：2019年12月

github：https://github.com/DrDavidS/basic_Machine_Learning

开源协议：[MIT](https://github.com/DrDavidS/basic_Machine_Learning/blob/master/LICENSE)

参考文献：

- 《深度学习入门》，作者：斋藤康毅；
- 《深度学习》，作者：Ian Goodfellow 、Yoshua Bengio、Aaron Courville。
- [Keras overview](https://tensorflow.google.cn/guide/keras/overview)
- [PyTorch Tutorials](https://pytorch.org/tutorials/)
- [CS231n](http://cs231n.stanford.edu/)

## 本节目的

在[3.02 神经网络的训练](https://github.com/DrDavidS/basic_Machine_Learning/blob/master/%E6%B7%B1%E5%BA%A6%E5%AD%A6%E4%B9%A0%E5%9F%BA%E7%A1%80/3.02%20%E7%A5%9E%E7%BB%8F%E7%BD%91%E7%BB%9C%E7%9A%84%E8%AE%AD%E7%BB%83.ipynb)中，我们简单介绍了神经网络是如何学习的，然后通过数值微分的方法计算了神经网络的损失函数 $L$ 关于权重参数 $W$ 的梯度，也就是 $\cfrac{\partial L}{\partial W}$。数值微分的方式简单，实现方便，但是微分操作在计算效率上很低。所以我们现在会学习一个能够高效计算梯度的方法 —— **反向传播(back propagation)算法**。

为了更精确和清楚地描述反向传播法算法，我们会采用基于**计算图（computational graph）**的形式。计算图描述会更加地直观，然后结合代码加深理解。

## 计算图

计算图将计算过程用图的形式表现出来，这里的图形就是数据结构图，通过多个节点和边表示。为了熟悉计算图，我们先用计算图解一下简单的，然后从这些简单问题出发，逐步深入，最终抵达误差反向传播法。

### 使用计算图求解

首先我们使用一个非常简单的例子作为开始。这个例子即使心算也能解答，这里的目的主要是让大家熟悉计算图。明白计算图的原理后，我们会在后面的复杂计算中看到它所发挥的巨大作用。

>首先给大家讲一讲经济常识，**价外税**的概念。增值税是一种价外税，价外税税款不包含在商品价格内的税，它是按照税收与价格的组成关系对税收进行的分类。实际上，为了方便起见，我们买的绝大多数商品的价格已经包含了增值税，以现行的增值税税率，水果的税率在13%。

>**假定**：
>
>这里我们假定，有这么一家超市，超市里面商品的标价都是**税前价格**。
>
>为了方便计算，我们再假定增值税的税率为10%，不过请记住，现在的大多数商品增值税税率是13%，少部分商品是9%。
>
> **问题1**：我在超市买了 2个 税前原价 100块 一个的苹果，苹果的增值税是 10% ，请计算我实际应该付多少钱？

计算图通过节点和箭头来表示计算过程。节点中是计算的**内容**，比如加法或者乘法之类，计算的**中间结果**写在箭头的上方。

![img](https://github.com/DrDavidS/basic_Machine_Learning/blob/master/back_up_images/%E8%AE%A1%E7%AE%97%E5%9B%BE1.jpg?raw=true)

如上图所示，苹果的 100元 流动到了 “$\times2$” 节点，变成 200元，然后传递到下一个节点。然后这个 200元 流向 “$\times1.1$” 节点，变成 220元。因此，从这个计算图可以知道，我应该付 220元。

实际上 “$\times2$”和“$\times1.1$” 等节点作为一个运算整体用圆圈 $○$ 括起来了，不过我们也可以只用 $○$ 来表示乘法运算 $\times$。

如下图所示，我们可以将 “2” 和 “1.1” 分别作为变量 “苹果的个数” 和 “增值税” 标在 $○$ 外面：

![img](https://github.com/DrDavidS/basic_Machine_Learning/blob/master/back_up_images/%E8%AE%A1%E7%AE%97%E5%9B%BE2.jpg?raw=true)

再看下一题：

>**问题2**：我在超市买了 2个 税前原价 100块 一个的苹果，以及 3个 税前原件 150块 一个的橘子，增值税是 10% ，请计算我实际应该付多少钱？

同上面的问题，我们还是用计算图来解决问题2，过程如下图：

![img](https://github.com/DrDavidS/basic_Machine_Learning/blob/master/back_up_images/%E8%AE%A1%E7%AE%97%E5%9B%BE3.jpg?raw=true)

在这个问题中我们新增了加法节点 “$+$” ，用来计算苹果和橘子的合计金额。构建了计算图后，我们从左到右进行计算，就向电路中的电流流动，计算结果也从左边传递到了右边。

到达最右边的计算结果以后，计算过程就结束了，从图中可以知道，问题2的答案是715元。

综上两个简单例子，我们总结一下计算图解题的流程情况：

>1. 构建计算图
>
>2. 在计算图上，从左向右进行计算。

这里第2步 “从左向右进行计算” 正是我们之前学习过的**前向传播（forward propagation）**。前向传播也叫正向传播，是从计算图出发点到结束点的传播。既然有前向传播，自然我们会想到有反向的情况，从图上就是从右到左。这种传播就是**反向传播（backward propagation）**。反向传播会在导数计算中发挥重要作用。

### 局部计算

计算图的特点就是可以通过传递 **“局部计算”** 获得最终结果。“局部” 的意思就是 “与自己相关的某个小范围”，所以局部计算就是指，无论全局发了什么，都能只根据与自己相关的信息输出接下来的结果。

![img](https://github.com/DrDavidS/basic_Machine_Learning/blob/master/back_up_images/%E8%AE%A1%E7%AE%97%E5%9B%BE4-%E6%9B%B4%E6%96%B0.jpg?raw=true)

如上图所示，假设（经过了复杂的计算）我们购买了很多其他的东西，总共花了4000元。这里的重点在于，节点处的计算都是局部的计算，这表明，例如苹果和其他很多东西的求和运算 （$4000+200\to4200$）并不关心 4000 这个数字是怎么计算而来的，只要简单相加就行了。换言之，各个节点处只需要进行与自己有关的计算，不用考虑全局。

综上，计算图可以集中精力于局部计算。无论全局的计算有多么复杂，各个步骤要做的就是对象节点的局部计算。虽然局部计算简单，但是通过传播它的计算结果，可以获得全局的复杂计算的结果，就像汽车厂里面的流水制造线一样。

### 为何使用计算图

前面我们用计算图举了两个很简单的例子，解决了两个问题。那么计算图的优点到底有什么？

一是刚刚提到的局部计算，无论全局是多么复杂，都可以通过局部计算使得各个节点致力于简单的计算，从而简化问题。

二是利用计算图可以保存中间的计算结果，比如刚刚我们计算到苹果和橘子等的分别购买的金额。

三则是最重要的原因，可以**通过反向传播高效计算导数**。

回到最开始的**问题1**，我们一开始计算的是，购买了 2 个苹果加上增值税后最终需要支付的金额。如果我们想要知道，有一天苹果涨价了，那么会在多大程度上影响最终支付的金额呢？

这相当于求 “支付金额关于苹果价格的导数”。假设苹果的价格是 $x$，支付金额为 $L$，则相当于求 $\cfrac{\partial L}{\partial x}$，这个导数的值就表示当苹果价格上涨的时候，支付金额会变多少。

>这里的支付金额函数 $L$有 3 个变量，一个是苹果价格，设为 $x$，另一个是税率，设为 $t$，苹果的数量设为 $n$，故有
>
>$$\large L = n\times x \times t$$
>
>所以，如果想求苹果价格变化导致最终支付结果变化的幅度，求的是 $L$ 关于 $x$ 的偏导数。

如前所述， “支付金额关于苹果价格的导数”的值可以通过计算图的反向传播求出来，至于反向传播具体怎么求，稍后就会介绍。先看看结果：

![img](https://github.com/DrDavidS/basic_Machine_Learning/blob/master/back_up_images/%E5%8F%8D%E5%90%91%E4%BC%A0%E6%92%AD0.jpg?raw=true)

如上图，反向传播以橙色箭头表示。反向传播传递“局部导数”，将导数的值写在箭头下方。

在这里，反向传播从右到左传递导数的值（$1\to 1.1\to 2.2$）。从这个结果可以知道 “支付金额关于苹果价格的导数”的值为 2.2。换句话说，苹果价格每上涨 1 元，最终支付的金额会多 2.2 元。

>再严格一点，如果苹果的价格增加一个微小值 $\Delta h$，那么最终支付的金额就会增加 $2.2\Delta h$。

这里我们求了 “支付金额关于苹果价格的导数”，同理我们也可以求 “支付金额关于苹果数量的导数”、“支付金额关于增值税的导数”，而且计算中途得到的导数结果（及中间传递的导数）可以被共享，从而可以高效地计算多个导数。

总结一下，计算图的优点就是可以通过正向传播和反向传播高效地计算各个变量的导数。

## 链式法则

前面的计算图主要讲的就是结果的前向传播，和我们平时接触的计算过程相符合，所以感觉上很自然。而反向传播将局部导数向正方向的反方向（从右到左）传递，可能会让人觉得困惑。

其实，传递这个局部导数的原理，是基于**链式法则（chain rule）**的，接下来我们会介绍链式法则，然后阐明其是如何对应计算图上的反向传播。

### 计算图的反向传播

首先看一个例子，假设存在一个 $y=f(x)$ 的计算，其反向传播如图：

![img](https://github.com/DrDavidS/basic_Machine_Learning/blob/master/back_up_images/%E5%8F%8D%E5%90%91%E4%BC%A0%E6%92%AD1.jpg?raw=true)

如上图所示，反向传播的计算顺序是，将信号 $E$ 乘以节点的局部导数 $\cfrac{\partial y}{\partial x}$，然后将结果传递给下一个节点。这里所说的局部导数是指正向传播中 $y=f(x)$ 的导数，也就是 $y$ 关于 $x$ 的导数 $\cfrac{\partial y}{\partial x}$。

比如，假设 $y=f(x)=x^2$，则局部导数 $\cfrac{\partial y}{\partial x} = 2x$。把这个局部导数乘以上游传过来的值（这里是 $E$ ），然后传递给前面的节点。

这就是反向传播的过程。通过这样的计算，我们可以高效地求出导数的值，这是反向传播的要点。至于具体实现，我们需要从链式法则说起。

### 链式法则简介

首先，我们回顾一下**复合函数**。

复合函数是由多个函数构成的函数，比如 $z={(x+y)}^2$ 是由下面两个式子构成的：

$$
\large 
\begin{equation}\begin{split} 
z&=t^2\\
t&=x+y
\end{split} \end{equation}
 \tag{5.1}$$

链式法则是关于复合函数导数的性质，定义：

- 如果某个函数由符合函数表示，则该复合函数的导数可以用构成复合函数的各个函数的导数的乘积表示。

实际上这是一个非常简单的性质，以上面的函数为例，$\cfrac{\partial z}{\partial x}$（$z$ 关于 $x$ 的导数）可以用 $\cfrac{\partial z}{\partial t}$ （$z$ 关于 $t$ 的导数）和 $\cfrac{\partial t}{\partial x}$ （$t$ 关于 $x$ 的导数）的乘积来表示。

用数学式表示，就是：

$$\large \frac{\partial z}{\partial x}=\frac{\partial z}{\partial t}\frac{\partial t}{\partial x} \tag{5.2}$$

然后其中两个 $\partial t$ 相互消除，就成了

$$\large \require{cancel} \frac{\partial z}{\partial x}=\frac{\partial z}{\cancel {\partial t}}\frac{\cancel{\partial t}}{\partial x}$$

现在我们使用链式法则来求式 5.2 的 $\cfrac{\partial z}{\partial x}$，首先我们要求偏导：

$$
\large 
\begin{equation}\begin{split} 
\frac{\partial z}{\partial t} &= 2t\\
\frac{\partial t}{\partial x} &= 1
\end{split} \end{equation}
 \tag{5.3}$$

如式 5.3 所示，$\cfrac{\partial z}{\partial t}$ 等于 $2t$，$\cfrac{\partial t}{\partial x}$ 等于 $1$。这是基于导数公式的解析解。然后最后要计算的$\cfrac{\partial z}{\partial x}$ 可以根据链式法则，计算导数乘积而得：

$$\large \frac{\partial z}{\partial x}=\frac{\partial z}{\partial t}\frac{\partial t}{\partial x}=2t\cdot1=2(x+y) \tag{5.4}$$

### 链式法则和计算图

现在我们尝试将式 5.4 的链式法则的计算用计算图来表示，其中平方运算我们采用 “$**2$” 代替：

![img](https://github.com/DrDavidS/basic_Machine_Learning/blob/master/back_up_images/%E5%8F%8D%E5%90%91%E4%BC%A0%E6%92%AD2.1.jpg?raw=true)

如图所示，计算图的反向传播从右到传播信号。反向传播的顺序是，先将节点的输入信号乘以节点的局部导数（偏导数），然后再传递给下一个节点。

比如，